In [43]:
from checkreq import FindImport, FindreqLibraries, changereqfile
from utils import GetFilesInDirectory

In [44]:
list_of_libs = []

for dir in GetFilesInDirectory('src'):
    list_of_libs += FindImport(dir)

In [45]:
list_of_libs

['datetime',
 'asyncio',
 'base64',
 'requests',
 'time',
 'langchain',
 'json',
 'boto3',
 'streamlit']

In [46]:
for dir in GetFilesInDirectory('src'):
    changereqfile("requirements2.txt", 'requirements.txt', dir)
    print(FindreqLibraries("requirements.txt", dir))

['requests @ file:///private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_b143dxenr7/croot/requests_1716902835977/work', 'requests-file @ file:///Users/ktietz/demo/mc3/conda-bld/requests-file_1629455781986/work', 'requests-oauthlib==1.3.1', 'requests-toolbelt @ file:///Users/builder/cbouss/perseverance-python-buildout/croot/requests-toolbelt_1699238632371/work']
['langchain==0.1.16', 'langchain-aws==0.1.2', 'langchain-cohere==0.1.5', 'langchain-community==0.0.34', 'langchain-core==0.1.46', 'langchain-openai==0.0.2.post1', 'langchain-text-splitters==0.0.1', 'json5 @ file:///tmp/build/80754af9/json5_1624432770122/work', 'jsonpatch @ file:///private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_3ajyoz8zoj/croot/jsonpatch_1714483362270/work', 'jsonpointer==2.1', 'jsonref==1.1.0', 'jsonschema @ file:///Users/builder/cbouss/perseverance-python-buildout/croot/jsonschema_1701803145350/work', 'jsonschema-specifications @ file:///Users/builder/cbouss/perseverance-python-buildout/cro

In [6]:
import dspy
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from typing import Annotated, Any, Dict, Optional, Sequence, TypedDict, List, Tuple

/Users/anilrao/miniforge3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [35]:
#Uses aws credentials from aws cli
bedrock = dspy.Bedrock(region_name="us-east-1")

model_kwargs =  { 
        "max_tokens": 8000,
        "temperature": 0.4,
        # "top_k": 50,
        # "top_p": 0.9
    }

lm = dspy.AWSMistral(bedrock, "mistral.mistral-large-2402-v1:0", **model_kwargs)

In [36]:
dspy.configure(lm=lm)

In [38]:
class Input(BaseModel):
    python_version: str = Field(description="Python version to be used")
    type_of_application: str = Field(description="Type of python application")
    work_directory: str = Field(description="Working directory of the application")
    entrypoint_filename: str = Field(description="File name for starting the application endpoint")

class Output(BaseModel):
    dockerfile: str = Field(description="The complete dockerfile code")


class CreateDockerFile(dspy.Signature):
    """I need a Dockerfile for a basic Streamlit app. The app is written in Python."""

    input: Input = dspy.InputField()
    output: Output = dspy.OutputField()


In [39]:
predictor = dspy.TypedPredictor(CreateDockerFile)
input = Input(
    python_version = "3.11",
    type_of_application = 'Streamlit',
    work_directory = 'src',
    entrypoint_filename = 'app.py'
)
mapped_parameters = predictor(input=input)

In [40]:
print(mapped_parameters.output.dockerfile)

FROM python:3.11
WORKDIR /src
COPY . /src
RUN pip install --no-cache-dir streamlit
CMD ["streamlit", "run", "app.py"]


In [41]:
with open("Dockerfile", "w") as file:
    file.write(mapped_parameters.output.dockerfile)